# a quick duckdb fts assistive technology search application

we take the results saved from [<time>2025-01-29</time>](2025-01-29-ravelry.ipynb) document
that searches from assistive technology on ravelry. 

warning: the search application is not is accessible to screen readers.

In [1]:
%% 
## initializing the database
load ravelry patterns into a dataframe from a the <var>file</var> on disk.
<details><summary>seed the <var>duckdb</var> database</summary>

    import ipywidgets, duckdb
    df = pandas.read_feather(file := "patterns.feather")
reuse and modify code from [Testing Out DuckDB's Full Text Search Extension](https://duckdb.org/2021/01/25/full-text-search.html)
to create our search application.

    atexit.register((
        connection := duckdb.connect(read_only=False)
    ).close)
create a `duckdb` database from the <var>df</var> the dataframe
and create the full text search components.

    connection.register('documents_df', df)
    connection.execute\
```sql
CREATE TABLE documents AS (SELECT * FROM documents_df);
```

    connection.execute\
```sql
PRAGMA create_fts_index('documents', 'id', 'notes', stopwords='english');
```

    connection.execute\
```sql
PREPARE fts_query AS (
    WITH scored_docs AS (
        SELECT *, fts_main_documents.match_bm25(id, ?) AS score FROM documents)
    SELECT permalink, notes
    FROM scored_docs
    WHERE score IS NOT NULL
    ORDER BY score DESC
    LIMIT 10)
```
    

</details>

initializing the database 
 load ravelry patterns into a dataframe from a the file on disk. 
 seed the duckdb database 
 import ipywidgets, duckdb
df = pandas.read_feather(file := "patterns.feather")
 
 reuse and modify code from Testing Out DuckDB's Full Text Search Extension 
to create our search application. 
 atexit.register((
 connection := duckdb.connect(read_only=False)
).close)
 
 create a duckdb database from the df the dataframe
and create the full text search components. 
 connection.register('documents_df', df)
connection.execute\
 
 CREATE TABLE documents AS ( SELECT * FROM documents_df ); 
 
 
 connection.execute\
 
 PRAGMA create_fts_index ( 'documents' , 'id' , 'notes' , stopwords = 'english' ); 
 
 
 connection.execute\
 
 PREPARE fts_query AS ( 
 WITH scored_docs AS ( 
 SELECT * , fts_main_documents . match_bm25 ( id , ? ) AS score FROM documents ) 
 SELECT permalink , notes 
 FROM scored_docs 
 WHERE score IS NOT NULL 
 ORDER BY score DESC 
 LIMIT 10 )

In [2]:
%%
<details><summary>the <var>search</var> function</summary>

    def search(q: str) -> DataFrame:
use `duckdb` full text search to find assistive technology patterns from ravelry.
        
        df = DataFrame(
            connection.execute("EXECUTE fts_query('" + q + "')").fetchall(), None, "permalink notes".split()
        )
        df.permalink = "https://www.ravelry.com/patterns/" + df.permalink 
        df = df.set_index("permalink")
        return df.notes.apply(shell.tangle.parser.parser.render).to_frame("info")

</details>

the search function 
 def search(q: str) -> DataFrame:
 
 use duckdb full text search to find assistive technology patterns from ravelry. 
 df = DataFrame(
 connection.execute("EXECUTE fts_query('" + q + "')").fetchall(), None, "permalink notes".split()
 )
 df.permalink = "https://www.ravelry.com/patterns/" + df.permalink 
 df = df.set_index("permalink")
 return df.notes.apply(shell.tangle.parser.parser.render).to_frame("info")

## demo

use <var>ipywidgets</var> to demo the search.

In [3]:
    @ipywidgets.interact
    def app(text: str = "hat"): return ipywidgets.HTML(search(text).style._repr_html_())

interactive(children=(Text(value='hat', description='text'), Output()), _dom_classes=('widget-interact',))